### DiBS marginal inference of $p(G | \mathcal{D})$ for linear Gaussian Bayes nets with BGe marginal likelihood

DiBS translates the task of inferring the posterior over Bayesian networks into an inference problem over the continuous latent variable $Z$. This is achieved by modeling the directed acyclic graph $G$ of the Bayesian network using the generative model $p(G | Z)$. The prior $p(Z)$ enforces the acyclicity of $G$.
Ultimately, this allows us to infer $p(G | \mathcal{D})$ or $p(G, \Theta | \mathcal{D})$ using off-the-shelf inference methods such as Stein Variational gradient descent (SVGD) (Liu and Wang, 2016).

In [ ]:
import jax
import jax.random as random

key = random.PRNGKey(123)
print(f"JAX backend: {jax.default_backend()}")

### Generate synthetic ground truth Bayesian network and BN model for inference

`data` contains information about and observations sampled from a synthetic, ground truth causal model with `n_vars` variables. By default, the conditional distributions are linear Gaussian. The random graph model is set by `graph_prior_str`, where `er` denotes Erdos-Renyi and `sf` scale-free graphs. 

`model` defines prior $p(G)$ and *marginal* likelihood $p(x | G)$ of the BN model for which DiBS will infer the posterior.

**For joint posterior inference, e.g., for inference of nonlinear Bayes net models, refer to the other example notebook `dibs_joint.ipynb`.** 


In [ ]:
from dibs.target import make_linear_gaussian_equivalent_model
from dibs.utils import visualize_ground_truth

key, subk = random.split(key)
data, model = make_linear_gaussian_equivalent_model(key=subk, n_vars=20, graph_prior_str="sf")

visualize_ground_truth(data.g)

### DiBS with SVGD

Infer $p(G | D)$ under the prior and marginal likelihood defined by `model`.
The below visualization shows the *matrix of edge probabilities* $G_\alpha(Z^{(k)})$ implied by each transported latent particle (i.e., sample) $Z^{(k)}$ during the iterations of SVGD with DiBS. Refer to the paper for further details.

To explicitly perform joint posterior inference of $p(G, \Theta | \mathcal{D})$ using a general likelihood $p(x | G, \Theta)$, use the separate, analogous class `JointDiBS`.



In [ ]:
%%time
from dibs.inference import MarginalDiBS

dibs = MarginalDiBS(x=data.x, interv_mask=None, inference_model=model)
key, subk = random.split(key)
gs = dibs.sample(key=subk, n_particles=20, steps=2000, callback_every=20, callback=dibs.visualize_callback())

### Evaluate on held-out data

Form the empirical (i.e., weighted by counts) and mixture distributions (i.e., weighted by unnormalized posterior probabilities, denoted DiBS+).

In [ ]:
dibs_empirical = dibs.get_empirical(gs)
dibs_mixture = dibs.get_mixture(gs)

Compute some evaluation metrics.

In [ ]:
from dibs.metrics import expected_shd, threshold_metrics, neg_ave_log_marginal_likelihood

for descr, dist in [('DiBS ', dibs_empirical), ('DiBS+', dibs_mixture)]:
    
    eshd = expected_shd(dist=dist, g=data.g)        
    auroc = threshold_metrics(dist=dist, g=data.g)['roc_auc']
    negll = neg_ave_log_marginal_likelihood(dist=dist, x=data.x_ho,
                eltwise_log_marginal_likelihood=dibs.eltwise_log_marginal_likelihood_observ)
    
    print(f'{descr} |  E-SHD: {eshd:4.1f}    AUROC: {auroc:5.2f}    neg. MLL {negll:5.2f}')
    